In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import gzip as gz

/home/usuario/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [69]:
#Encontrar o arquivo:

path = Path('/home/usuario/Área de Trabalho/Mestrado/dados/image')


filepaths = {}

mass = 'electron'
path_dir = Path(os.path.join(path, mass))
filepaths[mass] = pd.Series(list(os.listdir(path_dir)), name=f'{mass}').astype(str)

In [3]:
len(filepaths[mass])

830988

In [4]:
def getEvents(filepath):

    with gz.open(filepath, 'rb') as f:
      data = f.read().decode("utf-8") #The output reading Type is Byte, use .decode() to convert Byte to Str

    eventos = [line.split()[0] for line in data.split('\n')[:-1]]
    eventos = np.unique(eventos)

    return eventos

In [22]:
classe, telescopio, runs, eventos = [], [], [], []

for file in range(len(filepaths[mass])):
    tipo, a, tel, b, run, c, event = filepaths[mass][file].split('_')
    event = event[:-4]

    classe.append(tipo)
    telescopio.append(int(tel))
    runs.append(int(run))
    eventos.append(int(event))

In [23]:
sclasse = pd.Series(classe, name='massa')
stelescopio = pd.Series(telescopio, name='tel')
sruns = pd.Series(runs,name='run')
seventos = pd.Series(eventos,name='evento')

In [28]:
dataframe = pd.concat([filepaths[mass],sclasse, stelescopio, sruns, seventos], axis=1).sort_values('evento', ignore_index=True)

In [29]:
dataframe.head()

,electron,massa,tel,run,evento
0,electron_tel_2_run_356_event_100.txt,electron,2,356,100
1,electron_tel_2_run_340_event_100.txt,electron,2,340,100
2,electron_tel_3_run_364_event_100.txt,electron,3,364,100
3,electron_tel_1_run_340_event_100.txt,electron,1,340,100
4,electron_tel_1_run_364_event_100.txt,electron,1,364,100


In [31]:
summary = Path('/home/usuario/Área de Trabalho/Mestrado/dados/summary')

data = {}
data[mass] = {}
data_dir = os.path.join(summary, mass)

for num, telescope in enumerate(sorted(os.listdir(data_dir))):
    telescope_dir = os.path.join(data_dir, telescope)
    print(telescope_dir)
    data[mass][num+1] = pd.read_excel(telescope_dir, sheet_name = None)

/home/usuario/Área de Trabalho/Mestrado/dados/summary/electron/electron_tel_1.xlsx
/home/usuario/Área de Trabalho/Mestrado/dados/summary/electron/electron_tel_2.xlsx
/home/usuario/Área de Trabalho/Mestrado/dados/summary/electron/electron_tel_3.xlsx
/home/usuario/Área de Trabalho/Mestrado/dados/summary/electron/electron_tel_4.xlsx


In [32]:
header = data[mass][1]['run_1'].columns
for i in header:
  print(i)

  (1): event
  (2): telescope
  (3): energy
  (4): core distance to telescope
  (5): image size (amplitude) [p.e.]
  (6): number of pixels in image
  (7): width [deg.]
  (8): length [deg.]
  (9): distance [deg.]
 (10): miss [deg.]
 (11): alpha [deg.]
 (12): orientation [deg.]
 (13): direction [deg.]
 (14): image c.o.g. x [deg.]
 (15): image c.o.g. y [deg.]
 (16): Xmax [g/cm^2]
 (17): Hmax [m]
 (18): Npe (true number of photo-electrons)


In [75]:
def get_title(num:int, header):
  """
  Função para associar o título de cada coluna à seu respectivo número.

  num : int = número do feature do cabeçalho.
  header : list = cabeçalho do sumário.

  num = 3 -> energia
  num = 1 -> massa

  """

  return header[num-1]

def get_energy(massa, evento, datas, header):
  """
  Função que busca o valor da energia de um respectivo evento em uma run específica:

  feature_list : list = lista de features extraídos do nome do arquivo.
  data_df : pd.DataFrame = sumário da imagem que está sendo executada.
  name: str = nome do arquivo.

  Retorna o nome do arquivo com a energia em MeV.
  """

  df = datas

  #df = data_df[tel][run] #tem que mudar isso para ser automatico de acordo com cada

  if evento in df['  (1): event'].to_list():
    idx = df[get_title(1, header)].to_list().index(int(evento))
    return df[get_title(3, header)][3]
  else:
    return 0

In [56]:
header = data[mass][1]['run_1'].columns
energia = []

for file in range(len(filepaths[mass])):
    energy = get_energy(classe[file], eventos[file], data[mass][int(telescopio[file])][f'run_{int(runs[file])}'], header)
    energia.append(energy)

In [66]:
senergia = pd.Series(energia, name='energia')
mass_df = pd.concat([dataframe, senergia], axis=1)

In [68]:
mass

,electron,massa,tel,run,evento,energia
0,electron_tel_2_run_356_event_100.txt,electron,2,356,100,0.1573
1,electron_tel_2_run_340_event_100.txt,electron,2,340,100,0.2875
2,electron_tel_3_run_364_event_100.txt,electron,3,364,100,0.0434
3,electron_tel_1_run_340_event_100.txt,electron,1,340,100,1.3371
4,electron_tel_1_run_364_event_100.txt,electron,1,364,100,0.1531
...,...,...,...,...,...,...
830983,electron_tel_4_run_030_event_5000000.txt,electron,4,30,5000000,0.4528
830984,electron_tel_3_run_358_event_5000000.txt,electron,3,358,5000000,0.0536
830985,electron_tel_1_run_030_event_5000000.txt,electron,1,30,5000000,0.9820
830986,electron_tel_2_run_358_event_5000000.txt,electron,2,358,5000000,0.0759


In [70]:
mass_df.to_csv('/home/usuario/Área de Trabalho/Mestrado/dados/features/' + mass +'.csv', index=False)

In [64]:
mass.tail()

,electron,massa,tel,run,evento,energia
830983,electron_tel_4_run_030_event_5000000.txt,electron,4,30,5000000,0.4528
830984,electron_tel_3_run_358_event_5000000.txt,electron,3,358,5000000,0.0536
830985,electron_tel_1_run_030_event_5000000.txt,electron,1,30,5000000,0.9820
830986,electron_tel_2_run_358_event_5000000.txt,electron,2,358,5000000,0.0759
830987,electron_tel_1_run_051_event_5000000.txt,electron,1,51,5000000,0.0656


In [71]:
df_el = pd.read_csv('/home/usuario/Área de Trabalho/Mestrado/dados/features/electron.csv')

# Extrair features de todos os arquivos:

In [1]:
def get_title(num:int, header):
  """
  Função para associar o título de cada coluna à seu respectivo número.

  num : int = número do feature do cabeçalho.
  header : list = cabeçalho do sumário.

  num = 3 -> energia
  num = 1 -> massa

  """

  return header[num-1]

def get_energy(massa, evento, datas, header):
  """
  Função que busca o valor da energia de um respectivo evento em uma run específica:

  feature_list : list = lista de features extraídos do nome do arquivo.
  data_df : pd.DataFrame = sumário da imagem que está sendo executada.
  name: str = nome do arquivo.

  Retorna o nome do arquivo com a energia em MeV.
  """

  df = datas

  #df = data_df[tel][run] #tem que mudar isso para ser automatico de acordo com cada
  #print(df[get_title(1, header)])
  
  if int(evento) in df[get_title(1, header)].to_list():
    idx = df[get_title(1, header)].to_list().index(int(evento))
    return df[get_title(3, header)][3]
  else:
    return 0

In [5]:
#Encontrar o arquivo:

path = Path('/home/usuario/Área de Trabalho/Mestrado/dados/image')


filepaths = {}

mass = 'gamma_diffuse'
path_dir = Path(os.path.join(path, mass))
filepaths[mass] = pd.Series(list(os.listdir(path_dir)), name=f'{mass}').astype(str)

classe, telescopio, runs, eventos = [], [], [], []

for file in range(len(filepaths[mass])):
    tipo, a, tel, b, run, c, event = filepaths[mass][file].split('_')
    event = event[:-4]

    classe.append(tipo)
    telescopio.append(int(tel))
    runs.append(int(run))
    eventos.append(int(event))
    
sclasse = pd.Series(classe, name='massa')
stelescopio = pd.Series(telescopio, name='tel')
sruns = pd.Series(runs,name='run')
seventos = pd.Series(eventos,name='evento')

dataframe = pd.concat([filepaths[mass],sclasse, stelescopio, sruns, seventos], axis=1).sort_values('evento', ignore_index=True)

NameError: name 'Path' is not defined

In [23]:
len(filepaths['silicon'])

KeyError: 'silicon'

In [32]:
summary = Path('/home/usuario/Área de Trabalho/Mestrado/dados/summary')

data = {}
data[mass] = {}
data_dir = os.path.join(summary, mass)

for num, telescope in enumerate(sorted(os.listdir(data_dir))):
    telescope_dir = os.path.join(data_dir, telescope)
    print(telescope_dir)
    data[mass][num+1] = pd.read_excel(telescope_dir, sheet_name = None)

/home/usuario/Área de Trabalho/Mestrado/dados/summary/gamma/gamma_tel_1.xlsx
/home/usuario/Área de Trabalho/Mestrado/dados/summary/gamma/gamma_tel_2.xlsx
/home/usuario/Área de Trabalho/Mestrado/dados/summary/gamma/gamma_tel_3.xlsx
/home/usuario/Área de Trabalho/Mestrado/dados/summary/gamma/gamma_tel_4.xlsx


In [44]:
header = data[mass][1]['run_1'].columns
energia = []

for file in range(len(filepaths[mass])):
    #print(file)
    if f'run_{int(runs[file])}' in data[mass][telescopio[file]].keys():
        if int(runs[file]) == 2 and telescopio[file] == 2:
            energia.append(0)
        else:
            energy = get_energy(classe[file], eventos[file], data[mass][telescopio[file]][f'run_{int(runs[file])}'], header)
            energia.append(energy)

senergia = pd.Series(energia, name='energia')
mass_df = pd.concat([dataframe, senergia], axis=1)

mass_df.to_csv('/home/usuario/Área de Trabalho/Mestrado/dados/features/' + mass +'.csv', index=False)

In [80]:
df_el = pd.read_csv('/home/usuario/Área de Trabalho/Mestrado/dados/features/gamma.csv')

In [82]:
df_el.head()

,gamma,massa,tel,run,evento,energia
0,gamma_tel_3_run_093_event_100.txt,gamma,3,93,100,1.9753
1,gamma_tel_3_run_047_event_100.txt,gamma,3,47,100,0.0341
2,gamma_tel_3_run_040_event_100.txt,gamma,3,40,100,0.1156
3,gamma_tel_2_run_068_event_100.txt,gamma,2,68,100,0.4402
4,gamma_tel_3_run_083_event_100.txt,gamma,3,83,100,0.1310


In [7]:
get_title(3, header)

'  (3): energy'

In [8]:
header = data[mass][1]['run_1'].columns

In [26]:
for i in range(100):
    print(get_energy(classe[i], eventos[i], data["gamma"][telescopio[1]][f'run_{int(runs[i])}'], header))

0.4445
0.2444
0.1982
0
0.0991
0.1029
0.0353
0.0536
0.0977
0
4.0756
0.0991
2.6162
0.0349
0.2999
0.0977
0.0352
0.1878
0.5174
0.0728
0.0349
0
0.1982
1.9735
0
0.0369
0.1902
2.2546
0
0.0424
0.0876
0.5955
0.0363
0
0
0.0797
0.4141
0.4445
1.5329
0.0977
0.2109
1.5329
0
3.5624
0.1387
0.1982
0.0411
0.4759
0.0388
0.7582
0
4.9477
0.0643
1.1713
1.9753
0
0.0494
0
0.1722
0
1.5429
0.0349
0.0643
0.1023
0
0
0.3804
1.5429
0.4759
1.9753
3.9711
0.0797
0.0991
0.1156
0.0544
0.1028
0.0923
1.1068
2.2546
0.0424
1.0817
0.0536
1.9753
0.0806
0.0671
0.312
0.0388
1.674
0
0
0.0923
0.0671
0
3.9711
0
1.1068
2.0315
0.0911
1.1306
0.1878


In [35]:
telescopio[2959], runs[2959]

(2, 2)